In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import re
import statistics
from sklearn import tree
from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
import joblib

In [21]:
class SQLIAD:
    
    def carga_Dataset(self, dataframe):
        # El dataframe es creado con un dataset
        self.df= pd.read_csv(dataframe)
        
    def carga_Dato(self, dato):
        # Ingresar el dato que uno quiera que sea predecido
        # Crea un dataframe con un dato especifico
        # para que el modelo pueda hacer predicciones
        self.carga = [dato]
        self.df = pd.DataFrame()
        self.df['Sentence'] = None
        self.df['Sentence'] = self.carga
        
    def iden_Caract(self):
        self.F1 = self.df['Sentence'].str.contains('select ', case=False)
        self.F2 = self.df['Sentence'].str.contains('union', case=False)
        self.F3 = self.df['Sentence'].str.contains('update', case=False)
        self.F4 = self.df['Sentence'].str.contains('set', case=False)
        self.F5 = self.df['Sentence'].str.contains('alter', case=False)
        self.F6 = self.df['Sentence'].str.contains(' where ', case=False)
        self.F7 = self.df['Sentence'].str.contains('like', case=False)
        self.F8 = self.df['Sentence'].str.contains(' from ', case=False)
        self.F9 = self.df['Sentence'].str.contains(' table ', case=False)
        self.F10 = self.df['Sentence'].str.contains('database', case=False)
        nuevas_columnas1 = pd.DataFrame({'F1': self.F1, 'F2': self.F2, 'F3': self.F3, 'F4': self.F4, 'F5': self.F5, 
                                         'F6': self.F6, 'F7': self.F7, 'F8': self.F8, 'F9': self.F9, 'F10': self.F10})
        self.df = pd.concat([self.df, nuevas_columnas1], axis=1)
        self.F11 = self.df['Sentence'].str.contains('drop ', case=False)
        self.F12 = self.df['Sentence'].str.contains('delete ', case=False)
        self.F13 = self.df['Sentence'].str.contains('insert ', case=False)
        self.F14 = self.df['Sentence'].str.contains('And|Or', case=False, regex=True)
        self.F15 = self.df['Sentence'].str.contains('null', case=False)
        self.F16 = self.df['Sentence'].str.contains('=', case=False)
        self.F17 = self.df['Sentence'].str.contains('information_schema', case=False)
        self.F18 = self.df['Sentence'].str.contains('user', case=False)
        self.F19 = self.df['Sentence'].str.contains('version', case=False)
        self.F20 = self.df['Sentence'].str.contains('load_file', case=False)
        nuevas_columnas2 = pd.DataFrame({'F11': self.F11, 'F12': self.F12, 'F13': self.F13, 'F14': self.F14, 'F15': self.F15, 
                                         'F16': self.F16, 'F17': self.F17, 'F18': self.F18, 'F19': self.F19, 'F20': self.F20})
        self.df = pd.concat([self.df, nuevas_columnas2], axis=1)
        self.F21 = self.df['Sentence'].str.contains('save', case=False)
        self.F22 = self.df['Sentence'].str.contains('!|#|%|$|NUL|SOH|STX|ETX|EOT|@', case=False, regex=True)
        self.F23 = self.df['Sentence'].str.contains('&', case=False, regex=False)
        self.F24 = self.df['Sentence'].str.contains('|', case=False, regex=False)
        self.F25 = self.df['Sentence'].str.contains(',', case=False)
        self.F26 = self.df['Sentence'].str.contains(';', case=False)
        self.F27 = self.df['Sentence'].str.contains('\\', case=False, regex=False)
        self.F28 = self.df['Sentence'].str.contains('[-+*/]', case=False, regex=True)
        self.F29 = self.df['Sentence'].str.contains('commit|rollback|grant|revoke|declare|remove', case=False, regex=True)
        nuevas_columnas3 = pd.DataFrame({'F21': self.F21, 'F22': self.F22, 'F23': self.F23, 'F24': self.F24, 'F25': self.F25, 
                                         'F26': self.F26, 'F27': self.F27, 'F28': self.F28, 'F29': self.F29})
        self.df = pd.concat([self.df, nuevas_columnas3], axis=1)
        self.df['F30'] = self.df['Sentence'].str.count(';')
        self.grouping_name = 'F30'
        self.class_dic = {0: '0' , 1: '0'}
        self.df[self.grouping_name] = self.df[self.grouping_name].map(self.class_dic)
        self.df['F30'] = self.df['F30'].fillna('1')
        self.F31 = self.df['Sentence'].str.contains('/*', case=False, regex=False)
        self.F32 = self.df['Sentence'].str.contains('*/', case=False, regex=False)
        self.F33 = self.df['Sentence'].str.contains('\\x', case=False, regex=False)
        self.F34 = self.df['Sentence'].str.contains('\\u', case=False, regex=False)
        self.F35 = self.df['Sentence'].str.contains('connection', case=False)
        self.F36 = self.df['Sentence'].str.contains('xor', case=False)
        self.F37 = self.df['Sentence'].str.contains('inner join', case=False)
        self.F38 = self.df['Sentence'].str.contains('file|load_file|load_data_infile|into_outfile|into_dumpfile', case=False, regex=True)
        self.F39 = self.df['Sentence'].str.contains('OS|Operative System|exec', case=False, regex=True)
        self.F40 = self.df['Sentence'].str.count('STRING') + self.df['Sentence'].str.count('string')
        nuevas_columnas4 = pd.DataFrame({'F31': self.F31, 'F32': self.F32, 'F33': self.F33, 'F34': self.F34, 'F35': self.F35, 
                                         'F36': self.F36, 'F37': self.F37, 'F38': self.F38, 'F39': self.F39, 'F40': self.F40})
        self.df = pd.concat([self.df, nuevas_columnas4], axis=1)
        self.F41 = self.df['Sentence'].str.count('SUBSTR') + self.df['Sentence'].str.count('substr')
        self.F42 = self.df['Sentence'].str.count('SUBSTRING') + self.df['Sentence'].str.count('substring')
        self.F43 = self.df['Sentence'].str.count('MID') + self.df['Sentence'].str.count('mid')
        self.F44 = self.df['Sentence'].str.count('ASC') + self.df['Sentence'].str.count('asc')
        self.F45 = self.df['Sentence'].str.count('<')
        self.F46 = self.df['Sentence'].str.count('>')
        self.F47 = self.df['Sentence'].str.count('"')
        self.F48 = self.df['Sentence'].str.count("'")
        self.F49 = self.df['Sentence'].str.contains('exists', case=False)
        self.F50 = self.df['Sentence'].str.contains('floor', case=False)
        nuevas_columnas5 = pd.DataFrame({'F41': self.F41, 'F42': self.F42, 'F43': self.F43, 'F44': self.F44, 'F45': self.F45, 
                                         'F46': self.F46, 'F47': self.F47, 'F48': self.F48, 'F49': self.F49, 'F50': self.F50})
        self.df = pd.concat([self.df, nuevas_columnas5], axis=1)
        self.F51 = self.df['Sentence'].str.contains('rand', case=False)
        self.F52 = self.df['Sentence'].str.contains('group', case=False)
        self.F53 = self.df['Sentence'].str.contains('order', case=False)
        self.F54 = self.df['Sentence'].str.contains('lenght', case=False)
        self.F55 = self.df['Sentence'].str.contains('ascii', case=False)
        self.F56 = self.df['Sentence'].str.contains('if', case=False)
        self.F57 = self.df['Sentence'].str.contains('count', case=False)
        self.F58 = self.df['Sentence'].str.contains('sleep', case=False)
        self.F59 = self.df['Sentence'].str.contains('between', case=False)
        self.F60 = self.df['Sentence'].str.contains('values', case=False)
        nuevas_columnas6 = pd.DataFrame({'F51': self.F51, 'F52': self.F52, 'F53': self.F53, 'F54': self.F54, 'F55': self.F55, 
                                         'F56': self.F56, 'F57': self.F57, 'F58': self.F58, 'F59': self.F59, 'F60': self.F60})
        self.df = pd.concat([self.df, nuevas_columnas6], axis=1)
        self.F61 = self.df['Sentence'].str.contains('delay', case=False)
        self.F62 = self.df['Sentence'].str.contains('wait', case=False)
        self.F63 = self.df['Sentence'].str.contains('benchmark', case=False)
        self.F64 = self.df['Sentence'].str.contains('indentity_insert', case=False)
        self.F65 = self.df['Sentence'].str.contains('truncate table', case=False)
        self.F66 = self.df['Sentence'].str.contains('username | password', case=False, regex=True)
        self.F67 = self.df['Sentence'].str.contains('user | pass', case=False, regex=True)
        self.F68 = self.df['Sentence'].str.contains("')", case=False, regex=False)
        self.F69 = self.df['Sentence'].str.contains('limit', case=False)
        self.F70 = self.df['Sentence'].str.contains('concat', case=False)
        nuevas_columnas7 = pd.DataFrame({'F61': self.F61, 'F62': self.F62, 'F63': self.F63, 'F64': self.F64, 'F65': self.F65, 
                                         'F66': self.F66, 'F67': self.F67, 'F68': self.F68, 'F69': self.F69, 'F70': self.F70})
        self.df = pd.concat([self.df, nuevas_columnas7], axis=1)
        self.F71 = self.df['Sentence'].str.contains('ne', case=False)
        self.F72 = self.df['Sentence'].str.contains('find', case=False)
        self.F73 = self.df['Sentence'].str.contains('eq|gt|gte|lt|it|lte|ite|in|nin', case=False, regex=True)
        self.F74 = self.df['Sentence'].str.contains('mod| regex | text', case=False, regex=True)
        self.F75 = self.df['Sentence'].str.contains('return', case=False)
        self.F76 = self.df['Sentence'].str.contains('return true|return 1', case=False, regex=True)
        self.F77 = self.df['Sentence'].str.contains('exists', case=False)
        self.F78 = self.df['Sentence'].str.contains('create', case=False, regex=True)
        self.F79 = self.df['Sentence'].str.contains('show', case=False, regex=True)
        self.F80 = self.df['Sentence'].str.contains('collection', case=False, regex=True)
        self.F81 = self.df['Sentence'].str.contains('while(loop)', case=False, regex=False)
        nuevas_columnas8 = pd.DataFrame({'F71': self.F71, 'F72': self.F72, 'F73': self.F73, 'F74': self.F74, 'F75': self.F75, 
                                        'F76': self.F76, 'F77': self.F77, 'F78': self.F78, 'F79': self.F79, 'F80': self.F80,
                                        'F81': self.F81})
        self.df = pd.concat([self.df, nuevas_columnas8], axis=1)
        self.numro = 0
        self.Igualdad = []
        self.F83 = []
        self.Numeros = []
        self.Espacios = []
        self.Bytes = []
        self.Pyc = []
        self.Especial = []
        self.CantMayus = []
        self.LectEscrit = []
        self.Palabras_Clave = []
        #Patrón de palabras clave sql
        self.palabrasclave = ["select", "drop", "alter", "delete", "insert", "union", "update", "set", 
                                "where", "like", " from", "table", "between", "order", "join", "create"]
        self.byte_utf8max = []
        self.byte_utf8min = []
        self.byte_utf8DE = []
        for self.vx in self.df['Sentence']:
            self.partesPyc = self.df['Sentence'][self.numro].split(';') #Identifica si existe texto despues de un ;
            if len(self.partesPyc) > 1:
                self.Pyc.append('True')
            else:
                self.Pyc.append('False')
            self.texto =  [str(self.x) for self.x in self.df['Sentence']][self.numro].upper() #Identifica todos los caracteres
            self.F83.append(len(self.texto))
            self.Numeros.append(len([float(s) for s in re.findall(r'-?\d+\.?\d*', self.df['Sentence'][self.numro])])) #Identifica todos los números
            self.espac = 0
            for i, char in enumerate(self.df['Sentence'][self.numro]): #Identifica cantidad de espacios
                if char == ' ':
                    self.espac += 1
            self.Espacios.append(self.espac)
            self.Especial.append(len([str(s) for s in re.findall(r'[^\w\s]', self.df['Sentence'][self.numro])])) #Identifica todos los caracteres especiales
            self.baites = sys.getsizeof(self.df['Sentence'][self.numro]) #Identifica el peso en bytes del texto
            self.Bytes.append(self.baites)
            self.conmay = 0
            for char in self.df['Sentence'][self.numro]: #Identifica cuantas mayusculas hay en el texto
                if char.isupper():
                    self.conmay += 1
            self.CantMayus.append(self.conmay)
            patron_lectura = r'\b(?:{})\b'.format('|'.join(map(re.escape, ['select', ' where', 'order by', 'limit', 'join'])))
            patron_escritura = r'\b(?:{})\b'.format('|'.join(map(re.escape, ['create table', 'insert into', 'update', 'delete', 'alter table '])))
            #Identitica los patrones anteriores y clasifica si el texto contiene 
            #comandos de escritura en sql, comandos de lectura, ambos o ninguno
            if re.search(patron_lectura, self.df['Sentence'][self.numro], re.IGNORECASE) and re.search(patron_escritura, self.df['Sentence'][self.numro], re.IGNORECASE):
                self.LectEscrit.append(0)
            elif re.search(patron_lectura, self.df['Sentence'][self.numro], re.IGNORECASE) and not re.search(patron_escritura, self.df['Sentence'][self.numro], re.IGNORECASE):
                self.LectEscrit.append(1)
            elif re.search(patron_escritura, self.df['Sentence'][self.numro], re.IGNORECASE) and not re.search(patron_lectura, self.df['Sentence'][self.numro], re.IGNORECASE):
                self.LectEscrit.append(2)
            else:
                self.LectEscrit.append(3)
            #Verifica la cantidad de palabras clave de sql en el texto
            self.palcla = 0
            for palabra in self.palabrasclave:
                self.patronPC = r'\b' + re.escape(palabra) + r'\b'
                self.coincidencias = re.findall(self.patronPC, self.df['Sentence'][self.numro], re.IGNORECASE)
                self.palcla += len(self.coincidencias)
            self.Palabras_Clave.append(self.palcla)
            bytes_utf8 = []
            byte_utf8 = []
            for caracter in self.texto:
                bytes_utf8 = caracter.encode('utf-8') #Indica el caracter y el numero de byte en protocolo utf-8
                byte_utf8.append(len(bytes_utf8)) #Indica solo el número de byte
            self.byte_utf8max.append(max(byte_utf8))
            self.byte_utf8min.append(min(byte_utf8))
            self.lista = self.texto.split(None)
            self.numro = self.numro + 1
            self.elemento = '='  # elemento a buscar
            self.posiciones = []  # nuestra lista para guardar las posiciones
            self.posicion = -1  # empezaremos a buscar en posicion + 1 (que es 0 inicialmente)
            self.yuyu = 0
            try:
                while True:
                    # buscamos empezando a buscar desde la última posición encontrada
                    self.posicion = self.lista.index(self.elemento, self.posicion+1)
                    self.posiciones.append(self.posicion)
                    self.un = self.lista[self.posicion-1]
                    self.en = self.lista[self.posicion+1]
                    if self.en == self.un:
                        self.yuyu = 1
            except:
                pass  # no hacemos nada si index lanza ValueError
            if self.yuyu == 1:
                self.Igualdad.append('1')
            else:
                self.Igualdad.append('0')
        # Identifica una condicion siempre verdadera
        self.df['F82'] = self.Igualdad
        self.df['F82'] = self.df['F82'].astype(int)
        self.df['F83'] = self.F83 #Cantidad de caracteres en el texto
        self.df['F84'] = self.Numeros
        self.df['F85'] = self.df['F84']/self.df['F83'] #Relacion numeros y texto completo
        self.df['F86'] = self.Espacios
        self.df['F87'] = self.df['F86']/self.df['F83'] #Relacion espacios y texto completo
        self.df['F88'] = self.Especial
        self.df['F89'] = self.df['F88']/self.df['F83'] #Relacion caracteres especiales y texto completo
        #Cantidad de "texto normal" (no numeros, no caracteres especiales, no espacios)
        self.df['F90'] = self.df['F83']-(self.df['F84']+self.df['F86']+self.df['F88'])
        #Peso del texto en Bytes
        self.df['F91'] = self.Bytes
        #Si existe texto despues de un ;
        self.df['F92'] = self.Pyc
        #Lo convierte en booleano
        self.df['F92'] = self.df['F92'].astype(bool)
        #Verifica la cantidad de Mayusculas
        self.df['F93'] = self.CantMayus
        #Verifica si el comando es de lectura o escritura, o ninguno en caso de no ser comando sql
        self.df['F94'] = self.LectEscrit
        #Verifica si existen palabras clave de sql
        self.df['F95'] = self.Palabras_Clave
        #Número de bit maximo (F96) y minimo (F97) en formato utf8
        self.df['F96'] = self.byte_utf8max
        self.df['F97'] = self.byte_utf8min
        
        self.F98 = self.df['Sentence'].str.contains('\s{2,}', case=False, regex=True)
        self.F99 = self.df['Sentence'].str.contains('\dX', case=False, regex=True)
        self.F100 = self.df['Sentence'].str.contains('Select\s\d', case=False, regex=True)
        self.F101 = self.df['Sentence'].str.contains('version()', case=False, regex=False)
        self.F102 = self.df['Sentence'].str.contains('server_version|--version|-v|locate bin', case=False, regex=True)
        self.F103 = self.df['Sentence'].str.count('null')
        self.F104 = self.df['Sentence'].str.count('=')
        self.F105 = self.df['Sentence'].str.contains('query', case=False)
        self.F106 = self.df['Sentence'].str.contains('union all', case=False)
        self.F107 = self.df['Sentence'].str.contains('mysobjects|sysobjects', case=False, regex=True)
        self.F108 = self.df['Sentence'].str.contains('if', case=False)
        self.F109 = self.df['Sentence'].str.contains("'''", case=False)
        self.F110 = self.df['Sentence'].str.contains('Cast|Convert|Collate', case=False, regex=True)
        nuevas_columnas9 = pd.DataFrame({'F98': self.F98, 'F99': self.F99, 'F100': self.F100, 'F101': self.F101, 'F102': 
                                          self.F102, 'F103': self.F103, 'F104': self.F104, 'F105': self.F105, 'F106': self.F106, 
                                          'F107': self.F107, 'F108': self.F108, 'F109': self.F109, 'F110': self.F110})
        self.df = pd.concat([self.df, nuevas_columnas9], axis=1)
        self.F111 = self.df['Sentence'].str.contains('; set identity_insert', case=False, regex=False)
        self.F112 = self.df['Sentence'].str.contains('; truncate table', case=False, regex=False)
        self.F113 = self.df['Sentence'].str.contains('; drop table', case=False, regex=False)
        self.F114 = self.df['Sentence'].str.contains('; update', case=False, regex=False)
        self.F115 = self.df['Sentence'].str.contains('; insert into', case=False, regex=False)
        self.F116 = self.df['Sentence'].str.contains('; delete', case=False, regex=False)
        self.F117 = self.df['Sentence'].str.contains('; insert', case=False, regex=False)
        self.F118 = self.df['Sentence'].str.contains("''))/*&", case=False, regex=False)
        self.F119 = self.df['Sentence'].str.contains('/*&', case=False, regex=False)
        self.F120 = self.df['Sentence'].str.contains('And\s\d', case=False, regex=True)
        nuevas_columnas10 = pd.DataFrame({'F111': self.F111, 'F112': self.F112, 'F113': self.F113, 'F114': self.F114, 
                                          'F115': self.F115, 'F116': self.F116, 'F117': self.F117, 'F118': self.F118, 
                                          'F119': self.F119, 'F120': self.F120})
        self.df = pd.concat([self.df, nuevas_columnas10], axis=1)
        self.F121 = self.df['Sentence'].str.contains(';]/', case=False, regex=False)
        self.F122 = self.df['Sentence'].str.contains('createtable()', case=False, regex=False)
        self.F123 = self.df['Sentence'].str.contains('showtable()', case=False, regex=False)
        self.F124 = self.df['Sentence'].str.contains('createcollection()', case=False, regex=False)
        self.F125 = self.df['Sentence'].str.contains('drop()', case=False, regex=False)
        self.F126 = self.df['Sentence'].str.contains('dropdatabase()', case=False, regex=False)
        #Detecta si existe Or seguido de otra palabra
        self.F127 = self.df['Sentence'].str.contains(r'or\s+\S+', case=False, regex=True)
        self.F128 = self.df['Sentence'].str.contains(r'%\s+\S+', case=False, regex=True)
        #Detecta si existen Prepocisiones
        self.F129 = self.df['Sentence'].str.contains(' a | ante | bajo | cabe | con | contra | de | desde | durante | en | entre | hacia | hasta | mediante | para | por | según | sin | so | sobre | tras | versus | vía | aboard | about | above | across | after | against | along  | amid | among | anti | around | as | at | before | behindbelow | beneath | beside | besides | between | beyond | but | by | concerning | considering | despite | down | during | except | excepting | excluding | following | for | from | in | inside | into | like | minus | near | of | off | on | onto | opposite | outside | over | past | upon | versus | via | with | within | without ', case=False, regex=True)
        #Detecta si existen Articulos
        self.F130 = self.df['Sentence'].str.contains(' el | la | lo | los | las | un | una | unos | unas | a | an | the ', case=False, regex=True)
        nuevas_columnas11 = pd.DataFrame({'F121': self.F121, 'F122': self.F122, 'F123': self.F123, 'F124': self.F124, 
                                          'F125': self.F125, 'F126': self.F126, 'F127': self.F127, 'F128': self.F128, 
                                          'F129': self.F129, 'F130': self.F130})
        self.df = pd.concat([self.df, nuevas_columnas11], axis=1)
        
    def cargar_Caract(self):
        #Se seleccionan las caracterisitcas del modelo
        self.data = self.df[['Label','F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 
                      'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
                      'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30',
                      'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39', 'F40',
                      'F41', 'F42', 'F43', 'F44', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50',
                      'F51', 'F52', 'F53', 'F54', 'F55', 'F56', 'F57', 'F58', 'F59', 'F60',
                      'F61', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'F69', 'F70',
                      'F71', 'F72', 'F73', 'F74', 'F75', 'F76', 'F77', 'F78', 'F79', 'F80',
                      'F81', 'F82', 'F83', 'F84', 'F85', 'F86', 'F87', 'F88', 'F89', 'F90',
                      'F91', 'F92', 'F93', 'F94', 'F95', 'F96', 'F97', 'F98', 'F99', 'F100',
                      'F101', 'F102', 'F103', 'F104', 'F105', 'F106', 'F107', 'F108', 'F109', 'F110',
                      'F111', 'F112', 'F113', 'F114', 'F115', 'F116', 'F117', 'F118', 'F119', 'F120',
                      'F121', 'F122', 'F123', 'F124', 'F125', 'F126', 'F127', 'F128', 'F129', 'F130', 
                     ]]
        
    def carga_Features(self):
        # Guardamos solo las caracteristicas para que el modelo pueda hacer la prediccion
        self.Xy = self.df.drop(["Sentence"],axis = 1)
        
    def div_Datos(self):
        # Dividimos los datos en entrenamiento y prueba
        # X son nuestras variables independientes
        self.X = self.data.drop(["Label"],axis = 1)
        
        # y es nuestra variable dependiente
        self.y = self.data.Label
        
        # División 75% de datos para entrenamiento, 25% de datos para test
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, random_state=0)
        
    def k_Fold(self):
        # Dividimos los datos en entrenamiento y prueba
        # X son nuestras variables independientes
        self.X = self.data.drop(["Label"],axis = 1)
        
        # y es nuestra variable dependiente
        self.y = self.data.Label
        
        K = 5  # Puedes ajustar K según tus necesidades
        kf = KFold(n_splits=K, shuffle=True, random_state=0)
        
        scores = []
        
        for train_index, test_index in kf.split(self.X):
            self.X_train, self.X_test = self.X.iloc[train_index], self.X.iloc[test_index]
            self.y_train, self.y_test = self.y.iloc[train_index], self.y.iloc[test_index]
        
            # Crea y entrena un modelo de árbol de decisión
            self.model = tree.DecisionTreeClassifier(random_state=0)
            self.model.fit(self.X_train, self.y_train)
        
            # Realiza predicciones en el conjunto de prueba
            self.y_pred = self.model.predict(self.X_test)
        
            # Evalúa el rendimiento del modelo (puedes usar métricas como precisión, F1-score, etc.)
            accuracy = accuracy_score(self.y_test, self.y_pred)
            scores.append(accuracy)
            print('Precición de K-Fold por iteración', accuracy)
        
        # Calcula el promedio de las puntuaciones de precisión de los pliegues
        average_accuracy = np.mean(scores)
        print(f'Precisión promedio en {K}-Fold Cross-Validation: {average_accuracy}')

    def creacion_DT(self):
        # Creamos el modelo de Arbol de Decisión (y configuramos el número máximo de nodos-hoja)
        self.Dt_model = tree.DecisionTreeClassifier(random_state=0)#, max_leaf_nodes = 50)
    
    def entren_DT(self):
        # El modelo es entrenado
        self.Dt_model.fit(self.X_train, self.y_train)
        
    def accuracy_Entren(self):
        # Muestra el accuracy de los datos de entrenamiento
        print("Accuracy de entrenamiento: ", self.Dt_model.score(self.X_train, self.y_train))
        
    def f1_score_Entren(self):
        # Muestra el F1_score de los datos de entrenamiento
        self.y_pred = self.Dt_model.predict(self.X_train)
        print('F1 score:', f1_score(self.y_train, self.y_pred))
        
    def matriz_Conf_Ent(self):
        # Muestra la matriz de confusion de los datos de entrenamiento
        self.y_pred = self.Dt_model.predict(self.X_train)
        self.matriz = confusion_matrix(self.y_train,self.y_pred)

        plot_confusion_matrix(conf_mat=self.matriz, figsize=(4,4), show_normed=False)
        plt.tight_layout()
        
    def accuracy_Prueba(self):
        # Muestra el accuracy de los datos de prueba
        print("Accuracy de prueba: ", self.Dt_model.score(self.X_test, self.y_test))
        
    def f1_score_Prueba(self):
        # Muestra el F1_score de los datos de prueba
        self.y_pred = self.Dt_model.predict(self.X_test)
        print('F1 score:', f1_score(self.y_test, self.y_pred))
        
    def matriz_Conf_Prue(self):
        # Muestra la matriz de confusion de los datos de prueba
        self.y_pred = self.Dt_model.predict(self.X_test)
        self.matriz = confusion_matrix(self.y_test,self.y_pred)

        plot_confusion_matrix(conf_mat=self.matriz, figsize=(4,4), show_normed=False)
        plt.tight_layout()
        
    def carga_Modelo(self, modelo):
        # El modelo se carga
        # El modelo debe de ser un .pkl
        self.Dt_model = joblib.load(modelo)
        
    def prediccion(self):
        # El modelo hace la prediccion final
        self.pred = self.Dt_model.predict(self.Xy)
        #print(self.Xy)
        print('La predicción final fue: ',self.pred)
        
    def expDash(self):
        # Convierte las features a int y luego a float
        self.X = self.X.astype(int)
        self.X = self.X.astype(float)
        # Llama al explainer dashboard
        explainer = ClassifierExplainer(self.Dt_model, self.X, self.y)
        ExplainerDashboard(explainer).run(port=8051)
    
    def guardar_Modelo(self, nomMod):
        # Guarda el modelo entrenado
        # Es necesario ingresar el nombre del modelo y añadir ".plk" al final
        joblib.dump(self.Dt_model, nomMod)

# Entrenamiento del modelo

In [ ]:
dt = SQLIAD()
dt.carga_Dataset('Nombre_dataset.csv')
dt.iden_Caract()
dt.cargar_Caract()
dt.div_Datos()
dt.creacion_DT()
dt.entren_DT()
dt.accuracy_Entren()
dt.matriz_Conf_Ent()
dt.accuracy_Prueba()
dt.matriz_Conf_Prue()

# Comprobación con cross-validation (Accuracy)

In [ ]:
dtc = SQLIAD()
dtc.carga_Dataset('Nombre_dataset.csv')
dtc.iden_Caract()
dtc.cargar_Caract()
dtc.k_Fold()

# Predicción

In [ ]:
# dt = SQLIAD()
dt.carga_Dato('Dato a predecir')
dt.iden_Caract()
dt.carga_Features()
# dt.carga_Modelo('Nombre_modelo.pkl') # Esta linea solo es necesaria si quiere cargar un modelo
dt.prediccion()

# Guardar modelo

In [ ]:
dt.guardar_Modelo('Nombre_modelo.pkl')

# Cargar modelo

In [ ]:
dt.carga_Modelo('Nombre_modelo.pkl')

# Explainerdasboard

In [ ]:
dt.expDash()